In [ ]:
!pip install -U 'scikit-learn<0.24'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q keras

In [ ]:

import keras
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NER_dataset2_francais.csv", encoding="latin1").fillna(method="ffill")
df.tail(10)

,Unnamed: 0,company,address,name,phone
2808,2808,NOVATRANS,21 RUE ROCHER 75008 PARIS FRANCE,DENIS BRAUN,256673776
2809,2809,CILOMATE,AVENUE DE NANCY 54800 JARNY FRANCE,EMMANUEL BELLANGER,596470246
2810,2810,DOMINIQUE PRUDENT,BOIS CHIZE BP 28 71500 BRANGES FRANCE,TAHARY MARZOUK,406025736
2811,2811,DOMINIQUE PRUDENT,BOIS CHIZE BP 28 71500 BRANGES FRANCE,BAI UTTI,381787241
2812,2812,SOPITRA TRANSPORTS,AVENUE EUROPE - PA PETITES LANDES 44484 THOUAR...,ANA LUCIA SILVA,473088536
2813,2813,STAR'S SERVICE,1 BIS VILLA CHARLES 93800 EPINAY SUR SEINE FRANCE,LIONEL BATAILLE,628242867
2814,2814,DHL,"241, RUE BELLE ETOILE - ZI PARIS NORD II 56252...",LAURENT FAVAREILLE,460534880
2815,2815,DACHSER,ZI LES CHARTINIRES - PARC DES COSTIRES 01120 D...,TAMION FLORENCE,683873421
2816,2816,FRANCE BENNE,PA LES COLONNES - BOULEVARD ARAGO CS 50 79185 ...,OLIVIER LEVASSEUR,536488745
2817,2817,LOOMIS FRANCE,20 RUE MAURICE HENRI GUILBERT 94110 ARCUEIL FR...,M.ALQUIER,885567432


In [ ]:
import random

temp_data = [[],[],[]]

for i in range(len(df)):
    possibilites = ['company','address','name','phone']
    tags = ['C','Add','N','T']
    while(len(possibilites) != 0):
        n = random.randint(0,len(possibilites)-1)
        champs = possibilites[n]
        tag = tags[n]
        phrase = str(df[champs][i]).split()
        possibilites.pop(n)
        tags.pop(n)
        for j in range(0,len(phrase)):
            temp_data[0].append(str(i))
            temp_data[1].append(phrase[j]+' ')
            if(j==0):
                temp_data[2].append("B-"+tag)
            else:
                temp_data[2].append("I-"+tag)

data = pd.DataFrame({'Sentence #':temp_data[0],'Word':temp_data[1],'Tag':temp_data[2]})

In [ ]:
data

,Sentence #,Word,Tag
0,0,3,B-Add
1,0,Rue,I-Add
2,0,Adolphe,I-Add
3,0,Moehler,I-Add
4,0,-,I-Add
...,...,...,...
38024,2817,ARCUEIL,I-Add
38025,2817,FRANCE,I-Add
38026,2817,LOOMIS,B-C
38027,2817,FRANCE,I-C


In [ ]:
class SentenceGetter(object):

    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s['Word'].values.tolist(),  s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped['{}'.format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter = SentenceGetter(data)

In [ ]:
sentences = [[word[0] for word in sentence] for sentence in getter.sentences]
sentences[0]

['3 ',
 'Rue ',
 'Adolphe ',
 'Moehler ',
 '- ',
 'ZI ',
 'Nord ',
 '67210 ',
 'OBERNAI ',
 'France ',
 'TRANSPORT ',
 'HUCK ',
 'Mme ',
 'Denise ',
 'Scherer ',
 '307647897 ']

In [ ]:
labels = [[s[1] for s in sentence] for sentence in getter.sentences]
print(labels[0])

['B-Add', 'I-Add', 'I-Add', 'I-Add', 'I-Add', 'I-Add', 'I-Add', 'I-Add', 'I-Add', 'I-Add', 'B-C', 'I-C', 'B-N', 'I-N', 'I-N', 'B-T']


In [ ]:
tag_values = list(set(data["Tag"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [ ]:
!pip install transformers
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

torch.__version__

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 30.8 MB/s 
     |████████████████████████████████| 6.6 MB 66.6 MB/s 
     |████████████████████████████████| 86 kB 4.4 MB/s 
     |████████████████████████████████| 596 kB 61.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


'1.11.0+cu113'

In [ ]:
MAX_LEN = 75
bs = 32


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()


In [ ]:
torch.cuda.get_device_name(0)


'Tesla T4'

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)


Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels


In [ ]:
tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(sentences, labels)
]


In [ ]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]


In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")


In [ ]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")


In [ ]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]


In [ ]:
tr_inputs, temp_inputs, tr_tags, temp_tags = train_test_split(input_ids, tags,
                                                            random_state=2018, test_size=0.33)

val_inputs, test_inputs, val_tags, test_tags = train_test_split(temp_inputs, temp_tags,
                                                            random_state=2018, test_size=0.5)

tr_masks, temp_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.33)

val_masks, test_masks, _, _ = train_test_split(temp_masks, temp_inputs,
                                             random_state=2018, test_size=0.5)

print("total sample: " + str(len(input_ids)) + " train: "+str(len(tr_inputs))+" test & val: "+str(len(val_inputs)) )

total sample: 2818 train: 1888 test & val: 465


In [ ]:
"""
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)
"""

'\ntr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,\n                                                            random_state=2018, test_size=0.1)\ntr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,\n                                             random_state=2018, test_size=0.1)\n'

In [ ]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
test_inputs = torch.tensor(test_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
test_tags = torch.tensor(test_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)
test_masks = torch.tensor(test_masks)


In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

test_data = TensorDataset(test_inputs, test_masks, test_tags)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=bs)


In [ ]:
import transformers
from transformers import BertForTokenClassification, AdamW

transformers.__version__


'4.19.2'

In [ ]:
model = BertForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)


Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [ ]:
model.cuda();


In [ ]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 3
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)


In [ ]:
!pip install -q seqeval
from seqeval.metrics import f1_score, accuracy_score


     |████████████████████████████████| 43 kB 1.8 MB/s 


In [ ]:
## Store the average loss after each epoch so we can plot them.
loss_values, validation_loss_values = [], []

for _ in trange(epochs, desc="Epoch"):
    #####################Training

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0

    # Training loop
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # clear  previously calculated gradients
        model.zero_grad()
        # This will return the loss
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        # get the loss
        loss = outputs[0]
        # backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()
        #prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)

        optimizer.step()
        scheduler.step()

    # Calculate the average loss
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

    # Store the loss value
    loss_values.append(avg_train_loss)


    ############# Validation

    # Put the model into evaluation mode
    model.eval()
    # Reset the validation loss for this epoch.
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
        #Move logits and labels to CPU
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        #Calculate the accuracy
        eval_loss += outputs[0].mean().item()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

    eval_loss = eval_loss / len(valid_dataloader)
    validation_loss_values.append(eval_loss)
    print("Validation loss: {}".format(eval_loss))
    pred_tags = [[tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]]
    valid_tags = [[tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PAD"]]
    print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
    print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
    print()


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Average train loss: 0.649475363856655


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PAD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
Epoch:  33%|███▎      | 1/3 [00:25<00:51, 25.88s/it]

Validation loss: 0.2573206514120102
Validation Accuracy: 0.8289972089314195
Validation F1-Score: 0.6744418527157613

Average train loss: 0.20503961029699294


Epoch:  67%|██████▋   | 2/3 [00:52<00:26, 26.08s/it]

Validation loss: 0.1299956475694974
Validation Accuracy: 0.9234449760765551
Validation F1-Score: 0.8456501403180543

Average train loss: 0.12235994700152995


Epoch: 100%|██████████| 3/3 [01:19<00:00, 26.53s/it]

Validation loss: 0.10772769103447596
Validation Accuracy: 0.9350079744816587
Validation F1-Score: 0.8725946061950068



In [ ]:
test_preds = []
test_labels = []
for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():

            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)

        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # accuracy of batch 
        eval_loss += outputs[0].mean().item()
        test_preds.extend([list(p) for p in np.argmax(logits, axis=2)])
        test_labels.extend(label_ids)


In [ ]:
test_pred_tags = [tag_values[p_i] for p, l in zip(test_preds, test_labels)
                                 for p_i, l_i in zip(p, l) ]
test_true_tags = [tag_values[l_i] for l in test_labels
                                  for l_i in l]

In [ ]:
len(test_pred_tags)

34875

In [ ]:
len(tokenized_texts)

2818

In [ ]:
len(torch.tensor(input_ids))

2818

In [ ]:
len(test_inputs)

465

In [ ]:
len(test_true_tags)

34875

In [ ]:
tr_txt, temp_txt, tr_tags, temp_tags = train_test_split(tokenized_texts, tags,
                                                            random_state=2018, test_size=0.2)

val_txt, test_txt, val_tags, test_tags = train_test_split(temp_txt, temp_tags,
                                                            random_state=2018, test_size=0.5)

len(test_txt)

282

In [ ]:
len(test_txt)*75

21150

In [ ]:
len(test_txt)*75

21150

In [ ]:
test_txt_pred_label = list()

for i in range(0,len(test_txt)):
  for j in range(0,len(test_txt[i])):
    test_txt_pred_label.append([test_txt[i][j],test_pred_tags[i*75+j],test_true_tags[i*75 +j]])

with open('resultats_fine_tune_francais.txt', 'w') as f:
    for item in test_txt_pred_label:
        f.write("%s\n" % item)


In [ ]:
!pip install sklearn_crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 965 kB 39.7 MB/s 


In [ ]:
tag2idx_list = [k for (k,v) in tag2idx.items()]
print(tag2idx)
print(tag2idx_list)


{'B-N': 0, 'I-Add': 1, 'B-T': 2, 'B-Add': 3, 'I-C': 4, 'B-C': 5, 'I-N': 6, 'PAD': 7}
['B-N', 'I-Add', 'B-T', 'B-Add', 'I-C', 'B-C', 'I-N', 'PAD']


In [ ]:
test_labels_list = [item.tolist() for item in test_labels]


In [ ]:
test_labels_list_tag = [[tag2idx_list[e] for e in arr] for arr in test_labels_list]
test_labels_list_tag

[['B-C',
  'B-C',
  'B-C',
  'B-C',
  'B-C',
  'B-C',
  'I-C',
  'I-C',
  'I-C',
  'I-C',
  'B-N',
  'B-N',
  'B-N',
  'I-N',
  'I-N',
  'B-Add',
  'B-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'B-T',
  'B-T',
  'B-T',
  'B-T',
  'B-T',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD',
  'PAD'],
 ['B-C',
  'B-C',
  'B-C',
  'I-C',
  'I-C',
  'I-C',
  'I-C',
  'I-C',
  'I-C',
  'B-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'I-Add',
  'B-N',
  'B-N',
  'B-N',
  'I-N',
  'I-N',
  'I-N',
  'B-T',
  'B-

In [ ]:
test_preds_tag = [[tag2idx_list[e] for e in arr] for arr in test_preds]
test_preds_tag

In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
tag2idx_list.remove('PAD')

print(metrics.flat_classification_report(test_labels_list_tag, test_preds_tag, labels = tag2idx_list))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['B-N', 'I-Add', 'B-T', 'B-Add', 'I-C', 'B-C', 'I-N'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


              precision    recall  f1-score   support

         B-N       0.84      0.84      0.84      1676
       I-Add       0.99      0.99      0.99      8303
         B-T       1.00      1.00      1.00      2576
       B-Add       0.90      0.89      0.89       809
         I-C       0.86      0.87      0.86      2511
         B-C       0.91      0.79      0.84      2089
         I-N       0.82      0.91      0.86      2056

   micro avg       0.93      0.93      0.93     20020
   macro avg       0.90      0.90      0.90     20020
weighted avg       0.93      0.93      0.93     20020

